In [ ]:
from ldpc.codes import rep_code
from bposd.hgp import hgp
import numpy as np
from ldpc import bposd_decoder
from scipy.sparse import coo_matrix
from mqt.qecc import *
import time
from tqdm import tqdm

# 参数设置
ell = 15
m = 3
n = 2 * m * ell


# 定义循环移位矩阵
def cyclic_shift_matrix(size, shift):
    I = np.eye(size, dtype=int)
    return np.roll(I, shift, axis=1)


# 生成多项式 A 和 B 的循环矩阵
A1 = cyclic_shift_matrix(ell * m, 3)  # x^3
A2 = cyclic_shift_matrix(ell * m, 1)  # y^1
A3 = cyclic_shift_matrix(ell * m, 2)  # y^2
A = (A1 + A2 + A3) % 2  # A = x^3 + y^1 + y^2

B1 = cyclic_shift_matrix(ell * m, 3)  # y^3
B2 = cyclic_shift_matrix(ell * m, 1)  # x^1
B3 = cyclic_shift_matrix(ell * m, 2)  # x^2
B = (B1 + B2 + B3) % 2  # B = y^3 + x^1 + x^2

# 生成校验矩阵 H
Hx = np.hstack((A, B))
Hz = np.hstack((B.T, A.T))

# 打印校验矩阵 Hx 和 Hz
# print("Hx (X-check matrix):")
# print(Hx)
# print("\nHz (Z-check matrix):")
# print(Hz)

# UF decoder
code = Code(Hx, Hz)
uf_decoder = UFHeuristic()
uf_decoder.set_code(code)

# BP decoder
Hx_sparse = coo_matrix(Hx)
bpx_decoder = bposd_decoder(
    Hx_sparse,
    channel_probs=[None],
    max_iter=10000,
    bp_method="ms",
    ms_scaling_factor=0,
    osd_method="osd_cs",
    osd_order=7,
)

trials = 100

good_trials_bp = 0
bad_trials_bp = 0
good_trials_uf = 0
bad_trials_uf = 0

time_uf = 0
time_bp = 0

for i in tqdm(range(trials), desc="Running trials"):
    x_error = sample_iid_pauli_err(code.n, 0.05)
    x_syndrome = code.get_x_syndrome(x_error)

    """
    UF decoder
    """
    uf_start_time = time.time()
    uf_decoder.decode(x_syndrome)
    uf_end_time = time.time()
    result = uf_decoder.result
    residual_err_uf = np.array(x_error) ^ np.array(result.estimate)
    if code.is_x_stabilizer(residual_err_uf):
        good_trials_uf += 1
    else:
        bad_trials_uf += 1
    time_uf += uf_end_time - uf_start_time

    """
    BP+OSD decoder
    """
    x_syndrome_01 = np.array([int(x) for x in x_syndrome]).flatten()
    bp_start_time = time.time()
    bpx_decoder.decode(x_syndrome_01)
    bp_end_time = time.time()
    low_weight_error = bpx_decoder.osdw_decoding

    # 计算残余错误
    residual_err_bposd = (low_weight_error + x_error) % 2
    if code.is_x_stabilizer(residual_err_bposd):  # 使用 code.is_x_stabilizer
        good_trials_bp += 1
    else:
        bad_trials_bp += 1
    time_bp += bp_end_time - bp_start_time


print(f"good_trials_uf = {good_trials_uf}")
print(f"bad_trials_uf = {bad_trials_uf}")

print(f"good_trials_bp = {good_trials_bp}")
print(f"bad_trials_bp = {bad_trials_bp}")

print(f"average time_uf = {time_uf/trials}")
print(f"average time_bp = {time_bp/trials}")

Running trials:   0%|          | 0/100 [00:00<?, ?it/s]


AttributeError: 'ldpc.osd.bposd_decoder' object has no attribute 'time'